In [2]:
!pip install zodb

In [2]:
from mtgtools.MtgDB import MtgDB
mtg_db = MtgDB('my_db.fs')
mtg_db.scryfall_bulk_update()

Ignoring index for /Users/mateograciano/Desktop/magic_deckbuilding/my_db.fs


Attempting to update all the data from bulk...
querying Scryfall API for sets...
querying Scryfall API for bulk data...
-----------------------------------------------------------------------------------------------
Selected bulk type: "default_cards":  A JSON file containing every card object on Scryfall in English or the printed language if the card is only available in one language.
Updated at: 2024-08-18T21:13:12.983+00:00 
Size: 440 MB
-----------------------------------------------------------------------------------------------
Found a total of 920 new sets and 99239 new cards
Processing bulk data and updating cards.
-----------------------------------------------------------------------------------------------
Processing card: [82277 / 99239]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



The Scryfall database is now up to date! 
Elapsed time: 0:01:08

In [3]:
mtgio_cards = mtg_db.root.mtgio_cards

In [4]:
cards = mtg_db.root.scryfall_cards

In [5]:
d_cards = {}
for card in cards:
    if not card.name in d_cards:
        to_add = card.__dict__
        prices =[]
        to_add['usd_prices_all_versions'] = prices
        if to_add['prices']['usd']:
            to_add['usd_prices_all_versions'].append(to_add['prices']['usd'])
        d_cards[card.name] = to_add
    else:
        if card.__dict__['prices']['usd']:
            d_cards[card.name]['usd_prices_all_versions'].append(card.__dict__['prices']['usd'])
        

In [6]:
import numpy as np
def reduced_card_info(card):
    
    added_mana = ''
    if d_cards[card]['produced_mana']:
        added_mana = ','.join(d_cards[card]['produced_mana'])  
        
    image = None
    if d_cards[card]['image_uris']:
        image = d_cards[card]['image_uris']['normal']
    manacost = ''   
    if d_cards[card]['mana_cost']:
        manacost = d_cards[card]['mana_cost']
        
    try:
        link =d_cards[card]['related_uris']['gatherer']
    except:
        link =''
        
    try:
        link =d_cards[card]['related_uris']['edhrec']
    except:
        link =''
           

    line = [
    card,
    d_cards[card]['cmc'],
    np.median([ float(x) for x in d_cards[card]['usd_prices_all_versions']]),
    #d_cards[card]['oracle_text'],
    d_cards[card]['type_line'],
    image,
    #d_cards[card]['color_identity'],
    ','.join(d_cards[card]['color_identity']) , 
    added_mana,
    manacost,
    link
    ]

    df = pd.DataFrame(line).transpose()
    df.columns = ['card','cmc','price','card_type',
                  'image','color_identity','added_mana',
                  'manacost','link']

    return df
    #if 
import pandas as pd
dfs =[]

for card in d_cards:
    dfs.append(reduced_card_info(card))

/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/scrapper/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning

In [10]:
df[df.card=='Warren Soultrader']

,card,cmc,price,card_type,image,color_identity,added_mana,manacost,link
0,Warren Soultrader,3.0,6.82,Creature — Zombie Goblin Wizard,https://cards.scryfall.io/normal/front/1/7/17f...,B,"B,G,R,U,W",{2}{B},https://edhrec.com/route/?cc=Warren+Soultrader


In [7]:
df = pd.concat(dfs)
df.to_csv('game_info.csv')

df.head()

            
        

,card,cmc,price,card_type,image,color_identity,added_mana,manacost,link
0,Forest,0.0,0.25,Basic Land — Forest,https://cards.scryfall.io/normal/front/0/0/000...,G,G,,https://edhrec.com/route/?cc=Forest
0,Fury Sliver,6.0,0.28,Creature — Sliver,https://cards.scryfall.io/normal/front/0/0/000...,R,,{5}{R},https://edhrec.com/route/?cc=Fury+Sliver
0,Kor Outfitter,2.0,0.13,Creature — Kor Soldier,https://cards.scryfall.io/normal/front/0/0/000...,W,,{W}{W},https://edhrec.com/route/?cc=Kor+Outfitter
0,Spirit,0.0,0.155,Token Creature — Spirit,https://cards.scryfall.io/normal/front/0/0/000...,W,,,https://edhrec.com/route/?cc=Spirit
0,Siren Lookout,3.0,0.03,Creature — Siren Pirate,https://cards.scryfall.io/normal/front/0/0/000...,U,,{2}{U},https://edhrec.com/route/?cc=Siren+Lookout


In [85]:
int('c')

ValueError: invalid literal for int() with base 10: 'c'

'https://cards.scryfall.io/normal/front/0/6/06be8262-4636-4a2c-a0c8-de741cf45aed.jpg?1696638321'

In [114]:
','.join(d_cards[card]['color_identity'])

'R'

In [140]:
d_cards[card]

{'arena_id': None,
 'id': 'fff6ed1f-aaee-4f0e-b9f9-a377e66f9049',
 'lang': 'en',
 'mtgo_id': None,
 'mtgo_foil_id': None,
 'multiverse_ids': [],
 'tcgplayer_id': None,
 'tcgplayer_etched_id': None,
 'cardmarket_id': None,
 'oracle_id': 'bf84cf9a-6cc2-46d0-8c79-62ce3bd7542b',
 'prints_search_uri': 'https://api.scryfall.com/cards/search?order=released&q=oracleid%3Abf84cf9a-6cc2-46d0-8c79-62ce3bd7542b&unique=prints',
 'rulings_uri': 'https://api.scryfall.com/cards/fff6ed1f-aaee-4f0e-b9f9-a377e66f9049/rulings',
 'scryfall_uri': 'https://scryfall.com/card/asnc/12/a-little-chat-a-little-chat?utm_source=api',
 'uri': 'https://api.scryfall.com/cards/fff6ed1f-aaee-4f0e-b9f9-a377e66f9049',
 'all_parts': None,
 'card_faces': [{'object': 'card_face',
   'name': 'A Little Chat',
   'mana_cost': '',
   'type_line': 'Card',
   'oracle_text': '',
   'colors': [],
   'artist': 'Matt Stewart',
   'artist_id': '20871267-2d8a-41d5-b03a-be3d557c5734',
   'illustration_id': '4577c039-2ba7-493f-8563-c5c5f90e

In [28]:
cards[0].produced_mana
cards[0].image_uris

{'small': 'https://cards.scryfall.io/small/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
 'normal': 'https://cards.scryfall.io/normal/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
 'large': 'https://cards.scryfall.io/large/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
 'png': 'https://cards.scryfall.io/png/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.png?1562894979',
 'art_crop': 'https://cards.scryfall.io/art_crop/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
 'border_crop': 'https://cards.scryfall.io/border_crop/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979'}

In [30]:
cards[0].__dict__

{'arena_id': None,
 'id': '0000579f-7b35-4ed3-b44c-db2a538066fe',
 'lang': 'en',
 'mtgo_id': 25527,
 'mtgo_foil_id': 25528,
 'multiverse_ids': [109722],
 'tcgplayer_id': 14240,
 'tcgplayer_etched_id': None,
 'cardmarket_id': 13850,
 'oracle_id': '44623693-51d6-49ad-8cd7-140505caf02f',
 'prints_search_uri': 'https://api.scryfall.com/cards/search?order=released&q=oracleid%3A44623693-51d6-49ad-8cd7-140505caf02f&unique=prints',
 'rulings_uri': 'https://api.scryfall.com/cards/0000579f-7b35-4ed3-b44c-db2a538066fe/rulings',
 'scryfall_uri': 'https://scryfall.com/card/tsp/157/fury-sliver?utm_source=api',
 'uri': 'https://api.scryfall.com/cards/0000579f-7b35-4ed3-b44c-db2a538066fe',
 'all_parts': None,
 'card_faces': None,
 'cmc': 6.0,
 'color_identity': ['R'],
 'color_indicator': None,
 'colors': ['R'],
 'edhrec_rank': 7417,
 'hand_modifier': None,
 'keywords': [],
 'layout': 'normal',
 'legalities': {'standard': 'not_legal',
  'future': 'not_legal',
  'historic': 'not_legal',
  'timeless': 'n